In [ ]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Load and preprocess the dataset
def load_dataset(dataset_path):
    images = []
    labels = []
    class_names = sorted(os.listdir(dataset_path))
    count = 0
    for class_name in class_names:
        class_dir = os.path.join(dataset_path, class_name)
        for image_name in os.listdir('path_to_dataset'):
            image_path = os.path.join(dataset_path, image_name)
            image = cv2.imread(image_path)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            image = cv2.resize(image, (48, 48))
            images.append(image)
            labels.append(class_names.index(class_name))
        count += 1
        print(count)
    return np.array(images), np.array(labels)

# Preprocess the dataset
def preprocess_dataset(images, labels):
    images = images.reshape(-1, 48, 48, 1)
    images = images / 255.0
    labels = np.eye(853)[labels]  # Convert labels to one-hot encoding
    return images, labels

# Load and preprocess the dataset
dataset_path = 'path_to_dataset'
images, labels = load_dataset(dataset_path)
images, labels = preprocess_dataset(images, labels)

# Split the dataset into train and test sets
train_images, test_images, train_labels, test_labels = train_test_split(images, labels, test_size=0.2, random_state=42)

# Define the model architecture
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(48, 48, 1)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(853, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Create data generators for data augmentation
train_datagen = ImageDataGenerator(rotation_range=20, width_shift_range=0.1, height_shift_range=0.1,
                                   shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
train_datagen.fit(train_images)

# Train the model
model.fit(train_datagen.flow(train_images, train_labels, batch_size=32),
          steps_per_epoch=len(train_images) // 32, epochs=10)

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_images, test_labels)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)